In [1]:
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from collections import defaultdict
import random
import pickle
import word_preprocessing as wp

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /Users/binoy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
ml_classes = ['location', 'professor', 'time', 'professor_office_hours',
              'professor_office_location', 'project_details', 'syllabus',
              'exam_details', 'fallback']

processor  =  wp.Word_Processor('../grammar/', ml_classes)
train_X, train_Y, all_uniq_words = processor.words_to_vectors()
train_Y = train_Y.reshape(-1,1)

print(train_X.shape)
print(train_Y.shape)

(32022, 903)
(32022, 1)


In [3]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(train_X)
print(feature_columns)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[10, 10], n_classes=9, feature_columns=feature_columns, model_dir="./chat_model")
dnn_clf.fit(x=train_X, y=train_Y, steps=10000)


[_RealValuedColumn(column_name='', dimension=903, default_value=None, dtype=tf.int64, normalizer=None)]
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12ca8aac8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './chat_model'}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.

KeyboardInterrupt: 

In [ ]:
pickle.dump( {'all_uniq_words':all_uniq_words, 'uniq_ml_classes':ml_classes}, open( "training_data", "wb" ) )


In [ ]:
a = "where is class?"
a = "where is professor office?"
a = "yo when class start eh?"

a = "what is the project"
a = "what is the assignement on?"
a = "who is the president?"
a = "who is the professor"
a = "is it raining today?"
a = "who is stupid?"

x = processor.vectorize(a, all_uniq_words)

test_X = np.array(x).reshape(1, -1)

y_pred = list(dnn_clf.predict(test_X))
y_pred_proba = list(dnn_clf.predict_proba(test_X))
# print(test_X)
print(y_pred)
print(y_pred_proba)
print(ml_classes[y_pred[0]])

In [ ]:
print(all_uniq_words)